
# Ridge Regression Geometry Explorer

This notebook builds two interactive Bokeh visualizations that illuminate both the geometric and algebraic effects of ridge regression when features are highly correlated (here, correlation $r = 0.95$). Use the sliders to adjust the regularization strength $\lambda$ and watch how the model, the loss surface, and the associated Gram matrix respond.



## Setup
The following cell constructs a synthetic two-feature data set with strong multicollinearity, prepares helper functions, and pre-computes ridge solutions over a dense $\lambda$ grid. All subsequent visualizations reuse these cached values so that the interactivity remains smooth inside the notebook.


In [ ]:
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import (ColumnDataSource, ColorBar, LinearColorMapper, BasicTicker,
                          PrintfTickFormatter, Slider, CustomJS, Span, Div)
from bokeh.plotting import figure
from bokeh.palettes import Viridis256, Inferno256

np.random.seed(42)
output_notebook()

# Generate correlated features and response
n = 200
rho = 0.95
cov = np.array([[1.0, rho], [rho, 1.0]])
X = np.random.multivariate_normal(mean=[0, 0], cov=cov, size=n)
beta_true = np.array([3.0, -2.0])
noise = np.random.normal(scale=0.5, size=n)
y = X @ beta_true + noise

XtX = (X.T @ X) / n
Xty = (X.T @ y) / n

lambda_values = np.linspace(0.0, 5.0, 51)
num_lambdas = len(lambda_values)

beta_path = []
condition_numbers = []
min_eigs = []
max_eigs = []
eigen_spectra = []
identity = np.eye(X.shape[1])

for lam in lambda_values:
    gram_reg = XtX + lam * identity
    beta = np.linalg.solve(gram_reg, Xty)
    eigvals = np.linalg.eigvalsh(gram_reg)
    beta_path.append(beta)
    condition_numbers.append(np.max(eigvals) / np.min(eigvals))
    min_eigs.append(np.min(eigvals))
    max_eigs.append(np.max(eigvals))
    eigen_spectra.append(eigvals)

beta_path = np.array(beta_path)
condition_numbers = np.array(condition_numbers)
min_eigs = np.array(min_eigs)
max_eigs = np.array(max_eigs)
eigen_spectra = np.array(eigen_spectra)

b1_grid = np.linspace(-6, 6, 200)
b2_grid = np.linspace(-6, 6, 200)
B1, B2 = np.meshgrid(b1_grid, b2_grid, indexing="xy")

def ridge_loss_surface(lam):
    betas = np.stack([B1.ravel(), B2.ravel()], axis=1)
    preds = X @ betas.T
    residuals = y[:, None] - preds
    mse = np.mean(residuals ** 2, axis=0)
    penalty = lam * np.sum(betas ** 2, axis=1)
    return (mse + penalty).reshape(B1.shape)

loss_surfaces = []
contour_traces = []

for lam in lambda_values:
    surface = ridge_loss_surface(lam)
    loss_surfaces.append(surface.tolist())
    levels = np.linspace(surface.min(), surface.min() + 0.75 * (surface.max() - surface.min()), 6)
    cs = plt.contour(B1, B2, surface, levels=levels)
    xs, ys = [], []
    for collection in cs.collections:
        for path in collection.get_paths():
            verts = path.vertices
            xs.append(verts[:, 0].tolist())
            ys.append(verts[:, 1].tolist())
    contour_traces.append({"xs": xs, "ys": ys})
    plt.close()

loss_min = min(np.min(surface) for surface in loss_surfaces)
loss_max = max(np.max(surface) for surface in loss_surfaces)

path_source_data = dict(x=beta_path[:, 0], y=beta_path[:, 1])
opt_points = [beta_path[i] for i in range(num_lambdas)]

corr_matrix = np.corrcoef(X, rowvar=False)


## Loss Surface Visualization

The visualization below combines four complementary diagnostics:

1. **Loss Contours** (left): The MSE surface is rendered as a heatmap with overlaid contour lines. The optimal ridge coefficients trace the white path and the current optimum is highlighted in red.
2. **Coefficient Paths** (top-right): Tracks how $\beta_1$ and $\beta_2$ shrink toward the origin as $\lambda$ grows.
3. **Condition Number** (bottom-right): Reveals how ridge regularization rescales the loss surface, reducing the condition number and mitigating the ill-conditioning caused by correlated predictors.
4. **Live Diagnostics**: The dashboard reports the current $\lambda$, coefficients, and condition number so you can precisely monitor the trajectory.

Drag the slider slowly from 0 to 5 and observe how the loss surface roundness, the coefficient magnitudes, and the numerical stability evolve.


In [ ]:
# ColumnDataSources for interactive updates
image_source = ColumnDataSource(data=dict(image=[loss_surfaces[0]]))
contour_source = ColumnDataSource(data=contour_traces[0])
path_source = ColumnDataSource(data=path_source_data)
opt_source = ColumnDataSource(data=dict(x=[opt_points[0][0]], y=[opt_points[0][1]]))

beta_series_source = ColumnDataSource(data=dict(lambda=lambda_values,
                                                beta1=beta_path[:, 0],
                                                beta2=beta_path[:, 1]))
current_beta_source = ColumnDataSource(data=dict(lambda=[lambda_values[0]],
                                                 beta1=[beta_path[0, 0]],
                                                 beta2=[beta_path[0, 1]]))

condition_source = ColumnDataSource(data=dict(lambda=lambda_values, condition=condition_numbers))
condition_marker_source = ColumnDataSource(data=dict(lambda=[lambda_values[0]], condition=[condition_numbers[0]]))

loss_mapper = LinearColorMapper(palette=Viridis256, low=loss_min, high=loss_max)
loss_fig = figure(title="Ridge Loss Surface", x_axis_label="beta_1", y_axis_label="beta_2",
                  width=500, height=500, match_aspect=True)
loss_fig.image(image="image", x=b1_grid.min(), y=b2_grid.min(), dw=b1_grid.ptp(), dh=b2_grid.ptp(),
               color_mapper=loss_mapper, source=image_source)
loss_fig.multi_line(xs="xs", ys="ys", source=contour_source, line_color="white", line_alpha=0.8, line_width=1.5)
loss_fig.line("x", "y", source=path_source, line_color="white", line_alpha=0.6, line_width=2)
loss_fig.circle("x", "y", source=opt_source, size=12, color="red", line_color="white", line_width=1.5)
loss_color_bar = ColorBar(color_mapper=loss_mapper, ticker=BasicTicker(desired_num_ticks=8),
                          formatter=PrintfTickFormatter(format="%.2f"), location=(0, 0))
loss_fig.add_layout(loss_color_bar, "right")

coeff_fig = figure(title="Coefficient Paths vs Regularization", x_axis_label="lambda", y_axis_label="coefficient",
                   width=380, height=240)
coeff_fig.line("lambda", "beta1", source=beta_series_source, line_color="#1f77b4", line_width=3, legend_label="beta_1")
coeff_fig.line("lambda", "beta2", source=beta_series_source, line_color="#ff7f0e", line_width=3, legend_label="beta_2")
coeff_fig.circle("lambda", "beta1", source=current_beta_source, size=10, color="#1f77b4")
coeff_fig.circle("lambda", "beta2", source=current_beta_source, size=10, color="#ff7f0e")
coeff_span = Span(location=lambda_values[0], dimension="height", line_dash="dashed", line_width=2, line_color="black")
coeff_fig.add_layout(coeff_span)
coeff_fig.legend.location = "center_right"

cond_fig = figure(title="Condition Number Evolution", x_axis_label="lambda", y_axis_label="kappa( X^T X + lambda I )",
                  width=380, height=240, y_axis_type="log")
cond_fig.line("lambda", "condition", source=condition_source, line_width=3, color="#9467bd")
cond_fig.circle("lambda", "condition", source=condition_marker_source, size=10, color="#9467bd")
cond_span = Span(location=lambda_values[0], dimension="height", line_dash="dashed", line_width=2, line_color="black")
cond_fig.add_layout(cond_span)

info_div = Div(text=f"""
<b>lambda:</b> {lambda_values[0]:.2f}<br>
<b>beta_1:</b> {beta_path[0, 0]:.3f}<br>
<b>beta_2:</b> {beta_path[0, 1]:.3f}<br>
<b>Condition number:</b> {condition_numbers[0]:.2f}
""", width=380)

lambda_slider = Slider(title="lambda (ridge strength)", start=lambda_values[0], end=lambda_values[-1],
                       step=lambda_values[1] - lambda_values[0], value=lambda_values[0])

callback = CustomJS(args=dict(loss_images=loss_surfaces,
                              contours=contour_traces,
                              opt_points=opt_points,
                              image_source=image_source,
                              contour_source=contour_source,
                              opt_source=opt_source,
                              current_beta_source=current_beta_source,
                              coeff_span=coeff_span,
                              cond_span=cond_span,
                              condition_marker_source=condition_marker_source,
                              beta_path=beta_path,
                              lambda_values=lambda_values,
                              condition_numbers=condition_numbers,
                              info_div=info_div),
                    code="""
    const step = lambda_values[1] - lambda_values[0];
    const lam = cb_obj.value;
    const idx = Math.round((lam - lambda_values[0]) / step);

    image_source.data = {image: [loss_images[idx]]};
    contour_source.data = contours[idx];
    opt_source.data = {x: [opt_points[idx][0]], y: [opt_points[idx][1]]};

    current_beta_source.data = {
        lambda: [lambda_values[idx]],
        beta1: [beta_path[idx][0]],
        beta2: [beta_path[idx][1]]
    };

    condition_marker_source.data = {
        lambda: [lambda_values[idx]],
        condition: [condition_numbers[idx]]
    };

    coeff_span.location = lambda_values[idx];
    cond_span.location = lambda_values[idx];

    info_div.text = `<b>lambda:</b> ${lambda_values[idx].toFixed(2)}<br>` +
                    `<b>beta_1:</b> ${beta_path[idx][0].toFixed(3)}<br>` +
                    `<b>beta_2:</b> ${beta_path[idx][1].toFixed(3)}<br>` +
                    `<b>Condition number:</b> ${condition_numbers[idx].toFixed(2)}`;

    image_source.change.emit();
    contour_source.change.emit();
    opt_source.change.emit();
    current_beta_source.change.emit();
    condition_marker_source.change.emit();
""")

lambda_slider.js_on_change("value", callback)

layout = column(lambda_slider,
                row(loss_fig, column(coeff_fig, cond_fig, info_div)))

show(layout)


## Gram Matrix Visualization

To understand how ridge regression stabilizes linear models, inspect how the eigen-structure of the Gram matrix $X^\top X$ responds to the $\lambda I$ perturbation. This dashboard keeps the same $\lambda$ control but focuses on matrix diagnostics:

1. **Eigenvalue Spectrum**: Displays the two eigenvalues of $X^\top X + \lambda I$ on a log scale. Small eigenvalues lift quickly, illustrating how ridge acts most strongly where the matrix is nearly singular.
2. **Condition Number Path**: Re-emphasizes the drastic reduction in $\kappa$ for very small amounts of regularization.
3. **Min/Max Eigenvalue Tracking**: Shows how the eigen-gap narrows as $\lambda$ grows.
4. **Correlation Matrix Heatmap**: Highlights the strong $0.95$ correlation that creates the ill-conditioning in the first place.

Try $\lambda = 0.1$ to see how a minimal amount of shrinkage already improves the condition number by orders of magnitude.


In [ ]:
spectrum_source = ColumnDataSource(data=dict(x=np.arange(1, X.shape[1] + 1),
                                             y=eigen_spectra[0]))
condition_marker_source2 = ColumnDataSource(data=dict(lambda=[lambda_values[0]],
                                                      condition=[condition_numbers[0]]))
minmax_source = ColumnDataSource(data=dict(lambda=lambda_values,
                                           min_eig=min_eigs,
                                           max_eig=max_eigs))
minmax_marker_source = ColumnDataSource(data=dict(lambda=[lambda_values[0], lambda_values[0]],
                                                  eig=[min_eigs[0], max_eigs[0]],
                                                  label=["min", "max"]))

corr_mapper = LinearColorMapper(palette=Inferno256, low=0.0, high=1.0)
corr_source = ColumnDataSource(data=dict(image=[corr_matrix.tolist()]))

lambda_slider2 = Slider(title="lambda (ridge strength)", start=lambda_values[0], end=lambda_values[-1],
                        step=lambda_values[1] - lambda_values[0], value=lambda_values[0])

spectrum_fig = figure(title="Eigenvalue Spectrum of X^T X + lambda I", x_axis_label="eigenvalue index",
                      y_axis_label="eigenvalue", width=400, height=300, y_axis_type="log")
spectrum_fig.line("x", "y", source=spectrum_source, line_width=3, color="#17becf")
spectrum_fig.circle("x", "y", source=spectrum_source, size=10, color="#17becf")

cond_fig2 = figure(title="Condition Number vs lambda", x_axis_label="lambda", y_axis_label="kappa",
                   width=400, height=250, y_axis_type="log")
cond_fig2.line("lambda", "condition", source=condition_source, line_width=3, color="#bcbd22")
cond_fig2.circle("lambda", "condition", source=condition_marker_source2, size=10, color="#bcbd22")
cond_span2 = Span(location=lambda_values[0], dimension="height", line_dash="dashed", line_width=2, line_color="black")
cond_fig2.add_layout(cond_span2)

minmax_fig = figure(title="Min/Max Eigenvalues", x_axis_label="lambda", y_axis_label="eigenvalue",
                    width=400, height=250)
minmax_fig.line("lambda", "min_eig", source=minmax_source, line_width=3, color="#e377c2", legend_label="min eigenvalue")
minmax_fig.line("lambda", "max_eig", source=minmax_source, line_width=3, color="#7f7f7f", legend_label="max eigenvalue")
minmax_fig.circle(x="lambda", y="eig", source=minmax_marker_source, size=10,
                  color=["#e377c2", "#7f7f7f"])
minmax_span = Span(location=lambda_values[0], dimension="height", line_dash="dashed", line_width=2, line_color="black")
minmax_fig.add_layout(minmax_span)
minmax_fig.legend.location = "center_right"

corr_fig = figure(title="Feature Correlation Matrix", x_range=["x1", "x2"], y_range=["x2", "x1"],
                  width=320, height=320, toolbar_location=None)
corr_fig.image(image="image", x=0, y=0, dw=2, dh=2, color_mapper=corr_mapper, source=corr_source)
corr_fig.xaxis.major_label_overrides = {0.5: "x1", 1.5: "x2"}
corr_fig.yaxis.major_label_overrides = {0.5: "x2", 1.5: "x1"}
corr_color_bar = ColorBar(color_mapper=corr_mapper, location=(0, 0),
                          ticker=BasicTicker(desired_num_ticks=5), formatter=PrintfTickFormatter(format="%.2f"))
corr_fig.add_layout(corr_color_bar, "right")
corr_fig.xgrid.visible = False
corr_fig.ygrid.visible = False

info_div2 = Div(text=f"""
<b>lambda:</b> {lambda_values[0]:.2f}<br>
<b>Eigenvalues:</b> {', '.join(f'{val:.4f}' for val in eigen_spectra[0])}<br>
<b>Condition number:</b> {condition_numbers[0]:.2f}<br>
<b>Eigen-gap:</b> {(max_eigs[0] - min_eigs[0]):.4f}
""", width=400)

callback2 = CustomJS(args=dict(lambda_values=lambda_values,
                               eigen_spectra=eigen_spectra,
                               spectrum_source=spectrum_source,
                               condition_numbers=condition_numbers,
                               condition_marker_source=condition_marker_source2,
                               min_eigs=min_eigs,
                               max_eigs=max_eigs,
                               minmax_marker_source=minmax_marker_source,
                               cond_span=cond_span2,
                               minmax_span=minmax_span,
                               info_div=info_div2),
                     code="""
    const step = lambda_values[1] - lambda_values[0];
    const lam = cb_obj.value;
    const idx = Math.round((lam - lambda_values[0]) / step);

    const eigvals = eigen_spectra[idx];
    spectrum_source.data = {x: spectrum_source.data['x'], y: eigvals};
    spectrum_source.change.emit();

    condition_marker_source.data = {
        lambda: [lambda_values[idx]],
        condition: [condition_numbers[idx]]
    };
    condition_marker_source.change.emit();

    minmax_marker_source.data = {
        lambda: [lambda_values[idx], lambda_values[idx]],
        eig: [min_eigs[idx], max_eigs[idx]],
        label: ["min", "max"]
    };
    minmax_marker_source.change.emit();

    cond_span.location = lambda_values[idx];
    minmax_span.location = lambda_values[idx];

    info_div.text = `<b>lambda:</b> ${lambda_values[idx].toFixed(2)}<br>` +
                    `<b>Eigenvalues:</b> ${eigvals.map(v => v.toFixed(4)).join(', ')}<br>` +
                    `<b>Condition number:</b> ${condition_numbers[idx].toFixed(2)}<br>` +
                    `<b>Eigen-gap:</b> ${(max_eigs[idx] - min_eigs[idx]).toFixed(4)}`;
""")

lambda_slider2.js_on_change("value", callback2)

layout2 = column(lambda_slider2,
                 row(spectrum_fig, column(cond_fig2, minmax_fig, info_div2), corr_fig))

show(layout2)


## Exploration Tips

* **Loss Surface:** Move the slider slowly from 0 to 5. Notice how the optimum initially travels rapidly along the elongated valley before slowing as it nears the origin—an illustration of the bias–variance tradeoff.
* **Gram Matrix:** Even a tiny regularization (e.g., $\lambda = 0.1$) inflates the smallest eigenvalue dramatically, collapsing the condition number by several orders of magnitude.
* **Selective Shrinkage:** Because ridge adds $\lambda$ to every eigenvalue, the relative lift is largest for the tiniest eigenvalues. This selective stabilization is what tames multicollinearity without overly distorting well-identified directions.

Experiment with specific values (0, 0.1, 1, 3, 5) and cross-reference the two dashboards to see how geometric changes in parameter space mirror algebraic improvements in the Gram matrix.
